<a href="https://colab.research.google.com/github/RishitalakshmiG/Transformer-Based-Emotion-Recognition-Model/blob/main/Transformer_Based_Emotion_Recognition_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1
from google.colab import drive
drive.mount('/content/drive')
#working cell


In [ ]:
#2
path = "/content/drive/MyDrive/dailydialog.docx"
#working cell


In [ ]:
#3
!pip install python-docx pandas
#working cell


In [ ]:
#4
from google.colab import drive
drive.mount('/content/drive')

!ls -lh "/content/drive/MyDrive" | grep dailydialog
#working cell


In [ ]:
#5
from google.colab import drive
drive.mount('/content/drive')
#working cell


In [ ]:
#6
!ls "/content/drive/MyDrive"
#working cell


In [ ]:
#7
!ls "/content/drive/MyDrive/"

3. #Load the CSV with pandas:#
import pandas as pd

csv_path = "/content/drive/MyDrive/dailydialog.csv"  # change if inside folder
df = pd.read_csv(csv_path)

df.head()
#working cell

In [ ]:
#8
print(df.shape)        # dimensions
print(df.columns)      # column names
print(df.info())       # info and data types
print(df.isnull().sum())  # missing values count

#working cell

In [ ]:
#9
import pandas as pd
import ast
import re

# Suppose your dataframe is called data
# Fix the string so it can be converted to a Python list
def str_to_list(s):
    # Remove outer brackets if any
    s = s.strip()[1:-1]
    # Replace multiple spaces with comma
    s = re.sub(r"\s+", ",", s)
    # Convert to actual list of ints
    return [int(x) for x in s.split(",") if x]

# Apply to dialog, act, emotion columns
df['dialog']
#working cell


In [ ]:
#10
# Robust parser for dialog/act/emotion columns and flatten to utterance-level df
import ast, re
import pandas as pd
from typing import List
import nltk

# If you don't have nltk punkt tokenizer installed, uncomment:
# nltk.download('punkt')

def normalize_quotes(s: str) -> str:
    """Replace curly quotes and weird unicode apostrophes with ASCII equivalents."""
    if not isinstance(s, str):
        return s
    s = s.replace("“", '"').replace("”", '"').replace("’", "'").replace("‘", "'")
    s = s.replace("\u2013", "-").replace("\u2014", "-")
    return s

def try_literal_eval(s: str):
    """Try to parse a string with ast.literal_eval, safely returning None if it fails."""
    try:
        return ast.literal_eval(s)
    except Exception:
        return None

# Sentence splitter fallback: splits on punctuation (.!?)+ whitespace; preserves abbreviations badly but OK for dialogues.
sent_split_re = re.compile(r'(?<=[.!?])\s+')

def split_into_sentences_by_regex(text: str) -> List[str]:
    """Fallback: split a long string into sentences by punctuation-based regex."""
    if not isinstance(text, str):
        return []
    # normalize multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    parts = re.split(sent_split_re, text)
    # strip and remove empty parts
    parts = [p.strip() for p in parts if p.strip()]
    return parts

def parse_possible_dialog_field(raw):
    """
    Given raw value from 'dialog' column, return a list of utterances.
    Handles:
      - Proper python list string -> ast.literal_eval
      - Single-quoted long string -> split by sentence boundary
      - Already a list -> return as-is
    """
    if raw is None:
        return []
    # If it's already a list
    if isinstance(raw, list):
        return [str(x).strip() for x in raw if str(x).strip()]

    # Normalize quotes and whitespace first
    s = normalize_quotes(str(raw)).strip()

    # Try literal_eval directly (works if it's like "['utt1', 'utt2']" or "['utt1' 'utt2']" sometimes)
    parsed = try_literal_eval(s)
    if isinstance(parsed, list):
        return [str(x).strip() for x in parsed if str(x).strip()]

    # If literal_eval failed: check for pattern where numbers or tokens are space-separated like "[3 2 3]" etc.
    # For numeric lists (act/emotion) we'll handle elsewhere. For dialog, we check if s starts with [ and ends with ].
    if s.startswith('[') and s.endswith(']'):
        inner = s[1:-1].strip()
        # If inner contains quotes somewhere, try to extract quoted substrings
        quoted = re.findall(r"'(.*?)'|\"(.*?)\"", inner)
        # quoted is list of tuples: (group1, group2). extract non-empty and strip
        quoted_flat = [q[0] if q[0] else q[1] for q in quoted if (q[0] or q[1])]
        if quoted_flat:
            return [normalize_quotes(x).strip() for x in quoted_flat if x.strip()]
        # else: maybe it's a single long string (no commas / no separate quoted utterances)
        # treat inner as one long string and split into sentences
        return split_into_sentences_by_regex(inner)

    # If it doesn't look like bracketed list
    #working cell


In [ ]:
#11
import pandas as pd

csv_path = "/content/drive/MyDrive/dailydialog.csv"  # change if inside folder
df = pd.read_csv(csv_path)

print(df.head())
#working cell


In [ ]:
#12
# Robust parser for dialog/act/emotion columns and flatten to utterance-level df
import ast, re
import pandas as pd
from typing import List
import nltk

# If you don't have nltk punkt tokenizer installed, uncomment:
# nltk.download('punkt')

def normalize_quotes(s: str) -> str:
    """Replace curly quotes and weird unicode apostrophes with ASCII equivalents."""
    if not isinstance(s, str):
        return s
    s = s.replace("“", '"').replace("”", '"').replace("’", "'").replace("‘", "'")
    s = s.replace("\u2013", "-").replace("\u2014", "-")
    return s

def try_literal_eval(s: str):
    """Try to parse a string with ast.literal_eval, safely returning None if it fails."""
    try:
        return ast.literal_eval(s)
    except Exception:
        return None

# Sentence splitter fallback: splits on punctuation (.!?)+ whitespace; preserves abbreviations badly but OK for dialogues.
sent_split_re = re.compile(r'(?<=[.!?])\s+')

def split_into_sentences_by_regex(text: str) -> List[str]:
    """Fallback: split a long string into sentences by punctuation-based regex."""
    if not isinstance(text, str):
        return []
    # normalize multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    parts = re.split(sent_split_re, text)
    # strip and remove empty parts
    parts = [p.strip() for p in parts if p.strip()]
    return parts

def parse_possible_dialog_field(raw):
    """
    Given raw value from 'dialog' column, return a list of utterances.
    Handles:
      - Proper python list string -> ast.literal_eval
      - Single-quoted long string -> split by sentence boundary
      - Already a list -> return as-is
    """
    if raw is None:
        return []
    # If it's already a list
    if isinstance(raw, list):
        return [str(x).strip() for x in raw if str(x).strip()]

    # Normalize quotes and whitespace first
    s = normalize_quotes(str(raw)).strip()

    # Try literal_eval directly (works if it's like "['utt1', 'utt2']" or "['utt1' 'utt2']" sometimes)
    parsed = try_literal_eval(s)
    if isinstance(parsed, list):
        return [str(x).strip() for x in parsed if str(x).strip()]

    # If literal_eval failed: check for pattern where numbers or tokens are space-separated like "[3 2 3]" etc.
    # For numeric lists (act/emotion) we'll handle elsewhere. For dialog, we check if s starts with [ and ends with ].
    if s.startswith('[') and s.endswith(']'):
        inner = s[1:-1].strip()
        # If inner contains quotes somewhere, try to extract quoted substrings
        quoted = re.findall(r"'(.*?)'|\"(.*?)\"", inner)
        # quoted is list of tuples: (group1, group2). extract non-empty and strip
        quoted_flat = [q[0] if q[0] else q[1] for q in quoted if (q[0] or q[1])]
        if quoted_flat:
            return [normalize_quotes(x).strip() for x in quoted_flat if x.strip()]
        # else: maybe it's a single long string (no commas / no separate quoted utterances)
        # treat inner as one long string and split into sentences
        return split_into_sentences_by_regex(inner)

    # If it doesn't look like bracketed list at all, maybe it is just a multi-sentence string
    return split_into_sentences_by_regex(s)

def parse_numeric_list_field(raw):
    """
    Parse act/emotion field that might be:
      - already a list of ints
      - a string like '[1 2 3 4]' (space-separated)
      - a string like '[1,2,3,4]' (comma-separated)
      - a string like '1 2 3 4'
    Returns list of ints.
    """
    if raw is None:
        return []
    if isinstance(raw, list):
        # convert elements to ints if possible
        out = []
        for x in raw:
            try:
                out.append(int(x))
            except Exception:
                # try strip and then int
                try:
                    out.append(int(str(x).strip()))
                except Exception:
                    pass
        return out

    s = normalize_quotes(str(raw)).strip()
    # strip brackets
    if s.startswith('[') and s.endswith(']'):
        s = s[1:-1].strip()

    # try comma split first
    if ',' in s:
        parts = [p.strip() for p in s.split(',') if p.strip()]
    else:
        # split on whitespace
        parts = [p.strip() for p in re.split(r'\s+', s) if p.strip()]

    out = []
    for p in parts:
        try:
            out.append(int(p))
        except Exception:
            # try to remove any non-digit chars and parse
            p_clean = re.sub(r'[^\d-]', '', p)
            if p_clean:
                try:
                    out.append(int(p_clean))
                except:
                    pass
    return out

# Now apply parsing to your DataFrame 'data' safely and flatten
def flatten_dataframe_to_utterances(df):
    utterances = []
    emotions = []
    acts = []
    source_row_idx = []  # keep track of which conversation row they came from

    for idx, row in df.iterrows():
        raw_dialog = row['dialog']
        raw_act = row.get('act', None)
        raw_emotion = row.get('emotion', None)

        dialog_list = parse_possible_dialog_field(raw_dialog)
        act_list = parse_numeric_list_field(raw_act)
        emo_list = parse_numeric_list_field(raw_emotion)

        # If emotion/act lists are empty but dialog_list has sentences, we can either skip or assign -1
        if emo_list and len(emo_list) != len(dialog_list):
            # Attempt heuristics: sometimes emo_list is longer (multiple conversations collapsed) or shorter.
            # If emo_list length is 1 and dialog_list >1 -> assume same label for all utterances
            if len(emo_list) == 1:
                emo_list = emo_list * len(dialog_list)
            else:
                # if lengths mismatch, prefer length of dialog_list; pad with last label or -1
                if len(emo_list) < len(dialog_list):
                    # pad with last value
                    emo_list = emo_list + [emo_list[-1]] * (len(dialog_list) - len(emo_list))
                else:
                    # truncate emo_list
                    emo_list = emo_list[:len(dialog_list)]

        if act_list and len(act_list) != len(dialog_list):
            if len(act_list) == 1:
                act_list = act_list * len(dialog_list)
            else:
                if len(act_list) < len(dialog_list):
                    act_list = act_list + [act_list[-1]] * (len(dialog_list) - len(act_list))
                else:
                    act_list = act_list[:len(dialog_list)]

        # If emo_list is still empty, assign -1 placeholders
        if not emo_list:
            emo_list = [-1] * len(dialog_list)
        if not act_list:
            act_list = [-1] * len(dialog_list)

        for utt, a, e in zip(dialog_list, act_list, emo_list):
            utterances.append(utt)
            acts.append(a)
            emotions.append(e)
            source_row_idx.append(idx)

    out_df = pd.DataFrame({
        'utterance': utterances,
        'act': acts,
        'emotion': emotions,
        'source_idx': source_row_idx
    })
    return out_df

# Usage:
# data is your original DataFrame loaded from CSV
# safe: make a shallow copy if you want to keep original
parsed_df = flatten_dataframe_to_utterances(df)
print(parsed_df.head())
print("Counts:", parsed_df.shape)
# Quick checks
print(parsed_df['emotion'].value_counts(dropna=False).head())
#working cell


In [ ]:
#13
parsed_df = flatten_dataframe_to_utterances(df)
#working cell

In [ ]:
#14
print(parsed_df.head())
print(parsed_df.shape)
#working cell


In [ ]:
#15
import re

def clean_text(text):
    text = str(text).lower()                       # lowercase
    text = re.sub(r"[^a-zA-Z0-9\s']", " ", text)  # remove special chars, keep apostrophes
    text = re.sub(r"\s+", " ", text).strip()      # remove extra spaces
    return text

parsed_df['utterance_clean'] = parsed_df['utterance'].apply(clean_text)

# Quick check
print(parsed_df[['utterance', 'utterance_clean', 'emotion']].head())
#working cell

In [ ]:
#16
emotion_map = {0:'neutral',1:'anger',2:'disgust',3:'fear',4:'happy',5:'sad',6:'surprise'}
parsed_df['emotion_text'] = parsed_df['emotion'].map(emotion_map)

# Check distribution
print(parsed_df['emotion_text'].value_counts())
#working cell


In [ ]:
#17
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    parsed_df['utterance_clean'], parsed_df['emotion'],
    test_size=0.2,
    random_state=42,
    stratify=parsed_df['emotion']   # maintain class distribution
)
#working cell

In [ ]:
#18
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=64)
#working cell


In [ ]:
#19
import torch

train_labels_tensor = torch.tensor(list(train_labels))
test_labels_tensor = torch.tensor(list(test_labels))
#working cell


In [ ]:
# 20 training the model#

In [ ]:
#21
!pip install torch transformers scikit-learn
#working cell


In [ ]:
#22
#import libraries
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW  # Correct import
from sklearn.metrics import f1_score
#working cell

In [ ]:
#23
#creating a pytorch dataset
class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # returns input_ids, attention_mask, and label for each item
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create train and test datasets
train_dataset = EmotionDataset(train_encodings, train_labels_tensor)
test_dataset = EmotionDataset(test_encodings, test_labels_tensor)
#working cell


In [ ]:
#24
#creating dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)
#working cell


In [ ]:
#25
#initialising bert model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=7  # 7 emotion classes: neutral, anger, disgust, fear, happy, sad, surprise
)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
#working cell


In [ ]:
#26
#define optimiser
optimizer = AdamW(model.parameters(), lr=5e-5)
#working cell

In [ ]:
#27
#training loop
from tqdm import tqdm

epochs = 3

for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    total_loss = 0

    for batch in loop:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_description(f'Epoch {epoch+1}')
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} finished. Average Loss: {total_loss/len(train_loader):.4f}")
    #working cell


In [ ]:
model.eval()
preds = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute F1-score per class
from sklearn.metrics import classification_report

print(classification_report(true_labels, preds, target_names=[
    'neutral','anger','disgust','fear','happy','sad','surprise'
]))


In [ ]:
import matplotlib.pyplot as plt

parsed_df['emotion_text'].value_counts().plot(kind='bar', title='Emotion Distribution')
plt.show()


In [ ]:
# ===============================
# 1️⃣ Mount Google Drive & Load Dataset
# ===============================
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
csv_path = "/content/drive/MyDrive/dailydialog.csv"  # change if needed
df = pd.read_csv(csv_path)
print(df.head())
print(df.shape)

# ===============================
# 2️⃣ Preprocessing Utilities
# ===============================
import ast, re
from typing import List

def normalize_quotes(s: str) -> str:
    if not isinstance(s, str):
        return s
    s = s.replace("“", '"').replace("”", '"').replace("’", "'").replace("‘", "'")
    s = s.replace("\u2013", "-").replace("\u2014", "-")
    return s

def try_literal_eval(s: str):
    try:
        return ast.literal_eval(s)
    except Exception:
        return None

sent_split_re = re.compile(r'(?<=[.!?])\s+')
def split_into_sentences_by_regex(text: str) -> List[str]:
    if not isinstance(text, str):
        return []
    text = re.sub(r'\s+', ' ', text).strip()
    parts = re.split(sent_split_re, text)
    return [p.strip() for p in parts if p.strip()]

def parse_possible_dialog_field(raw):
    if raw is None:
        return []
    if isinstance(raw, list):
        return [str(x).strip() for x in raw if str(x).strip()]
    s = normalize_quotes(str(raw)).strip()
    parsed = try_literal_eval(s)
    if isinstance(parsed, list):
        return [str(x).strip() for x in parsed if str(x).strip()]
    if s.startswith('[') and s.endswith(']'):
        inner = s[1:-1].strip()
        quoted = re.findall(r"'(.*?)'|\"(.*?)\"", inner)
        quoted_flat = [q[0] if q[0] else q[1] for q in quoted if (q[0] or q[1])]
        if quoted_flat:
            return [normalize_quotes(x).strip() for x in quoted_flat if x.strip()]
        return split_into_sentences_by_regex(inner)
    return split_into_sentences_by_regex(s)

def parse_numeric_list_field(raw):
    if raw is None:
        return []
    if isinstance(raw, list):
        out = []
        for x in raw:
            try: out.append(int(x))
            except: pass
        return out
    s = normalize_quotes(str(raw)).strip()
    if s.startswith('[') and s.endswith(']'):
        s = s[1:-1].strip()
    parts = [p.strip() for p in re.split(r'[,\s]+', s) if p.strip()]
    out = []
    for p in parts:
        try: out.append(int(p))
        except:
            p_clean = re.sub(r'[^\d-]', '', p)
            if p_clean:
                try: out.append(int(p_clean))
                except: pass
    return out

def flatten_dataframe_to_utterances(df):
    utterances, emotions, acts, source_idx = [], [], [], []
    for idx, row in df.iterrows():
        dialog_list = parse_possible_dialog_field(row['dialog'])
        act_list = parse_numeric_list_field(row.get('act', None))
        emo_list = parse_numeric_list_field(row.get('emotion', None))

        if emo_list and len(emo_list) != len(dialog_list):
            if len(emo_list) == 1:
                emo_list *= len(dialog_list)
            else:
                if len(emo_list) < len(dialog_list):
                    emo_list += [emo_list[-1]] * (len(dialog_list)-len(emo_list))
                else:
                    emo_list = emo_list[:len(dialog_list)]

        if act_list and len(act_list) != len(dialog_list):
            if len(act_list) == 1:
                act_list *= len(dialog_list)
            else:
                if len(act_list) < len(dialog_list):
                    act_list += [act_list[-1]] * (len(dialog_list)-len(act_list))
                else:
                    act_list = act_list[:len(dialog_list)]

        if not emo_list: emo_list = [-1]*len(dialog_list)
        if not act_list: act_list = [-1]*len(dialog_list)

        for utt, a, e in zip(dialog_list, act_list, emo_list):
            utterances.append(utt)
            acts.append(a)
            emotions.append(e)
            source_idx.append(idx)

    return pd.DataFrame({
        'utterance': utterances,
        'act': acts,
        'emotion': emotions,
        'source_idx': source_idx
    })

parsed_df = flatten_dataframe_to_utterances(df)

# ===============================
# 3️⃣ Clean Text
# ===============================
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-zA-Z0-9\s']", " ", text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

parsed_df['utterance_clean'] = parsed_df['utterance'].apply(clean_text)

# Map emotion IDs to text
emotion_map = {0:'neutral',1:'anger',2:'disgust',3:'fear',4:'happy',5:'sad',6:'surprise'}
parsed_df['emotion_text'] = parsed_df['emotion'].map(emotion_map)

# ===============================
# 4️⃣ Train-Test Split
# ===============================
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    parsed_df['utterance_clean'], parsed_df['emotion'],
    test_size=0.2, random_state=42, stratify=parsed_df['emotion']
)

# ===============================
# 5️⃣ Tokenization (DistilBERT)
# ===============================
!pip install transformers torch scikit-learn --quiet
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=64)

import torch
train_labels_tensor = torch.tensor(list(train_labels))
test_labels_tensor = torch.tensor(list(test_labels))

# ===============================
# 6️⃣ PyTorch Dataset & DataLoader
# ===============================
from torch.utils.data import Dataset, DataLoader

class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = EmotionDataset(train_encodings, train_labels_tensor)
test_dataset = EmotionDataset(test_encodings, test_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # larger batch for speed
test_loader = DataLoader(test_dataset, batch_size=32)

# ===============================
# 7️⃣ Model & Optimizer
# ===============================
from transformers import DistilBertForSequenceClassification
from torch.optim import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=7
)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

# ===============================
# 8️⃣ Training Loop (Mixed Precision)
# ===============================
from tqdm import tqdm
scaler = torch.cuda.amp.GradScaler()
epochs = 3

for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    total_loss = 0
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.cuda.amp.autocast():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        loop.set_description(f'Epoch {epoch+1}')
        loop.set_postfix(loss=loss.item())
    print(f"Epoch {epoch+1} finished. Avg Loss: {total_loss/len(train_loader):.4f}")

# ===============================
# 9️⃣ Evaluation
# ===============================
model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(true_labels, preds)
print(f"Test Accuracy: {accuracy*100:.2f}%")
print(classification_report(true_labels, preds, target_names=[
    'neutral','anger','disgust','fear','happy','sad','surprise'
]))

# ===============================
# 10️⃣ Emotion Distribution Plot
# ===============================
import matplotlib.pyplot as plt
parsed_df['emotion_text'].value_counts().plot(kind='bar', title='Emotion Distribution')
plt.show()


In [ ]:
from sklearn.utils import resample

# 1️⃣ Separate majority and minority classes
dfs = []
target_count = 2000  # desired number of samples per class

for emotion in parsed_df['emotion_text'].unique():
    df_class = parsed_df[parsed_df['emotion_text'] == emotion]
    if len(df_class) < target_count:
        # Oversample minority class
        df_class_upsampled = resample(
            df_class,
            replace=True,
            n_samples=target_count,
            random_state=42
        )
    else:
        df_class_upsampled = df_class.sample(n=target_count, random_state=42)
    dfs.append(df_class_upsampled)

# 2️⃣ Combine all classes
df_balanced = pd.concat(dfs).reset_index(drop=True)

# 3️⃣ Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check distribution
print(df_balanced['emotion_text'].value_counts())


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_balanced['utterance_clean'], df_balanced['emotion'],
    test_size=0.2, random_state=42, stratify=df_balanced['emotion']
)

In [ ]:
# ===============================
# 1️⃣ Tokenization (DistilBERT)
# ===============================
from transformers import DistilBertTokenizer
import torch

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=64)

train_labels_tensor = torch.tensor(list(train_labels))
test_labels_tensor = torch.tensor(list(test_labels))

# ===============================
# 2️⃣ PyTorch Dataset & DataLoader
# ===============================
from torch.utils.data import Dataset, DataLoader

class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = EmotionDataset(train_encodings, train_labels_tensor)
test_dataset = EmotionDataset(test_encodings, test_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# ===============================
# 3️⃣ Model & Optimizer
# ===============================
from transformers import DistilBertForSequenceClassification
from torch.optim import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=7
)
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

# ===============================
# 4️⃣ Training Loop (Mixed Precision)
# ===============================
from tqdm import tqdm
scaler = torch.cuda.amp.GradScaler()
epochs = 3

for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    total_loss = 0
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.cuda.amp.autocast():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        loop.set_description(f'Epoch {epoch+1}')
        loop.set_postfix(loss=loss.item())
    print(f"Epoch {epoch+1} finished. Avg Loss: {total_loss/len(train_loader):.4f}")

# ===============================
# 5️⃣ Evaluation: Accuracy + F1 + Confusion Matrix
# ===============================
model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Accuracy
accuracy = accuracy_score(true_labels, preds)
print(f"Test Accuracy: {accuracy*100:.2f}%\n")

# F1-score per class
print("Classification Report:\n")
print(classification_report(true_labels, preds, target_names=[
    'neutral','anger','disgust','fear','happy','sad','surprise'
]))

# Confusion Matrix
cm = confusion_matrix(true_labels, preds)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d',
            xticklabels=['neutral','anger','disgust','fear','happy','sad','surprise'],
            yticklabels=['neutral','anger','disgust','fear','happy','sad','surprise'],
            cmap="Blues")
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

# Emotion Distribution Plot
plt.figure(figsize=(8,5))
df_balanced['emotion_text'].value_counts().plot(kind='bar', title='Balanced Emotion Distribution')
plt.show()


In [ ]:
#newcode

In [ ]:
from tqdm import tqdm
scaler = torch.cuda.amp.GradScaler()
epochs = 30

for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    total_loss = 0
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.cuda.amp.autocast():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        loop.set_description(f'Epoch {epoch+1}')
        loop.set_postfix(loss=loss.item())
    print(f"Epoch {epoch+1} finished. Avg Loss: {total_loss/len(train_loader):.4f}")


In [ ]:
# ===============================
# 5️⃣ Evaluation: Accuracy + F1 + Confusion Matrix - 30 epochs
# ===============================
model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)
        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Accuracy
accuracy = accuracy_score(true_labels, preds)
print(f"Test Accuracy: {accuracy*100:.2f}%\n")

# F1-score per class
print("Classification Report:\n")
print(classification_report(true_labels, preds, target_names=[
    'neutral','anger','disgust','fear','happy','sad','surprise'
]))

# Confusion Matrix
cm = confusion_matrix(true_labels, preds)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d',
            xticklabels=['neutral','anger','disgust','fear','happy','sad','surprise'],
            yticklabels=['neutral','anger','disgust','fear','happy','sad','surprise'],
            cmap="Blues")
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

# Emotion Distribution Plot
plt.figure(figsize=(8,5))
df_balanced['emotion_text'].value_counts().plot(kind='bar', title='Balanced Emotion Distribution')
plt.show()


In [ ]:
#conference

In [ ]:
#Install Dependencies
!pip install transformers torch scikit-learn seaborn --quiet


In [ ]:
#Imports'
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import (
    BertTokenizer, BertForSequenceClassification,
    DistilBertTokenizer, DistilBertForSequenceClassification,
    RobertaTokenizer, RobertaForSequenceClassification,
    AlbertTokenizer, AlbertForSequenceClassification,
    XLNetTokenizer, XLNetForSequenceClassification
)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

from tqdm import tqdm


In [ ]:
#Load Oversampled Dataset
# If previously saved:
# df_balanced = pd.read_csv("/content/drive/MyDrive/df_balanced.csv")

df_balanced.head()


In [ ]:
#Train/Test Split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_balanced["utterance_clean"],
    df_balanced["emotion"],
    test_size=0.2,
    random_state=42,
    stratify=df_balanced["emotion"]
)

train_labels = list(train_labels)
test_labels = list(test_labels)


In [ ]:
#Dataset Class
class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item


In [ ]:
#Universal Transformer Trainer (With Curves + Confusion Matrices)
def train_transformer(model_name, tokenizer, model_class, epochs):

    print(f"\n====================")
    print(f" Training {model_name} for {epochs} epochs")
    print(f"====================\n")

    # Tokenize
    train_enc = tokenizer(list(train_texts), truncation=True, padding=True, max_length=64)
    test_enc = tokenizer(list(test_texts), truncation=True, padding=True, max_length=64)

    train_dataset = EmotionDataset(train_enc, train_labels)
    test_dataset = EmotionDataset(test_enc, test_labels)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model_class.from_pretrained(model_name, num_labels=7).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

    # Stores
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for batch in tqdm(train_loader):
            optimizer.zero_grad()

            ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(ids, attention_mask=mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            loss.backward()
            optimizer.step()

        train_losses.append(total_loss / len(train_loader))
        train_accs.append(correct / total)

        # Validation
        model.eval()
        val_loss = 0
        v_correct = 0
        v_total = 0

        with torch.no_grad():
            for batch in test_loader:
                ids = batch["input_ids"].to(device)
                mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(ids, attention_mask=mask, labels=labels)
                val_loss += outputs.loss.item()
                preds = torch.argmax(outputs.logits, dim=1)

                v_correct += (preds == labels).sum().item()
                v_total += labels.size(0)

        val_losses.append(val_loss / len(test_loader))
        val_accs.append(v_correct / v_total)

        print(f"Epoch {epoch+1}: TrainLoss={train_losses[-1]:.4f}, ValLoss={val_losses[-1]:.4f}")

    # Evaluation
    preds, truth = [], []
    model.eval()
    with torch.no_grad():
        for batch in test_loader:
            ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            out = model(ids, attention_mask=mask)
            pred = torch.argmax(out.logits, dim=1)

            preds.extend(pred.cpu().numpy())
            truth.extend(labels.cpu().numpy())

    report = classification_report(
        truth, preds,
        output_dict=True,
        target_names=['neutral','anger','disgust','fear','happy','sad','surprise']
    )
    accuracy = accuracy_score(truth, preds)

    # Plots: Loss Curves
    plt.figure(figsize=(10,4))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.title(f"{model_name} Loss Curve ({epochs} epochs)")
    plt.legend()
    plt.show()

    # Accuracy Curves
    plt.figure(figsize=(10,4))
    plt.plot(train_accs, label="Train Acc")
    plt.plot(val_accs, label="Val Acc")
    plt.title(f"{model_name} Accuracy Curve ({epochs} epochs)")
    plt.legend()
    plt.show()

    # -----------------------------
    # CONFUSION MATRIX (Counts + Percentages)
    # -----------------------------
    classes = ['neutral','anger','disgust','fear','happy','sad','surprise']
    cm = confusion_matrix(truth, preds)

    # Raw counts
    plt.figure(figsize=(7,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title(f"{model_name} Confusion Matrix (Counts)")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

    # Percentages
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm_percent = np.round(cm_percent * 100, 2)

    plt.figure(figsize=(7,6))
    sns.heatmap(cm_percent, annot=True, fmt='.2f', cmap='Greens',
                xticklabels=classes, yticklabels=classes)
    plt.title(f"{model_name} Confusion Matrix (Percentages)")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

    return accuracy, report


In [ ]:
#LSTM/RNN/NN Models
class LSTMClassifier(nn.Module):
    def __init__(self, vocab=30000, embed_dim=128, hidden=128, num_classes=7):
        super().__init__()
        self.embed = nn.Embedding(vocab, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden, batch_first=True)
        self.fc = nn.Linear(hidden, num_classes)

    def forward(self, x):
        x = self.embed(x)
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1])

class RNNClassifier(nn.Module):
    def __init__(self, vocab=30000, embed_dim=128, hidden=128, num_classes=7):
        super().__init__()
        self.embed = nn.Embedding(vocab, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden, batch_first=True)
        self.fc = nn.Linear(hidden, num_classes)

    def forward(self, x):
        x = self.embed(x)
        _, h = self.rnn(x)
        return self.fc(h[-1])

class SimpleNN(nn.Module):
    def __init__(self, vocab=30000, embed_dim=128, num_classes=7):
        super().__init__()
        self.embed = nn.Embedding(vocab, embed_dim)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*128, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.embed(x)
        return self.fc(x)


In [ ]:
#Train Non-Transformer Models (LSTM/RNN/NN)
def train_simple_model(model, epochs):

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    train_ids = tokenizer(list(train_texts), truncation=True, padding='max_length',
                          max_length=64, return_tensors="pt")["input_ids"]
    test_ids  = tokenizer(list(test_texts), truncation=True, padding='max_length',
                          max_length=64, return_tensors="pt")["input_ids"]

    train_dataset = torch.utils.data.TensorDataset(train_ids, torch.tensor(train_labels))
    test_dataset  = torch.utils.data.TensorDataset(test_ids, torch.tensor(test_labels))

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader  = DataLoader(test_dataset, batch_size=32)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for x, y in tqdm(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            logits = model(x)
            loss = criterion(logits, y)

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

            loss.backward()
            optimizer.step()

        train_losses.append(total_loss / len(train_loader))
        train_accs.append(correct / total)

        # Validation
        model.eval()
        val_loss = 0
        v_correct = 0
        v_total = 0
        y_true = []
        y_pred = []

        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                logits = model(x)
                loss = criterion(logits, y)

                val_loss += loss.item()

                preds = torch.argmax(logits, dim=1)
                y_true.extend(y.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())

                v_correct += (preds == y).sum().item()
                v_total += y.size(0)

        val_losses.append(val_loss / len(test_loader))
        val_accs.append(v_correct / v_total)

        print(f"Epoch {epoch+1}: TrainLoss={train_losses[-1]:.4f}, ValLoss={val_losses[-1]:.4f}")

    # Loss curves
    plt.figure(figsize=(10,4))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.title(f"{model.__class__.__name__} Loss Curve ({epochs} epochs)")
    plt.legend()
    plt.show()

    # Accuracy curves
    plt.figure(figsize=(10,4))
    plt.plot(train_accs, label="Train Accuracy")
    plt.plot(val_accs, label="Val Accuracy")
    plt.title(f"{model.__class__.__name__} Accuracy Curve ({epochs} epochs)")
    plt.legend()
    plt.show()

    # -----------------------------
    # Confusion Matrix (Counts + Percentages)
    # -----------------------------
    classes = ['neutral','anger','disgust','fear','happy','sad','surprise']

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(7,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title(f"{model.__class__.__name__} Confusion Matrix (Counts)")
    plt.show()

    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm_percent = np.round(cm_percent * 100, 2)

    plt.figure(figsize=(7,6))
    sns.heatmap(cm_percent, annot=True, fmt='.2f', cmap='Greens',
                xticklabels=classes, yticklabels=classes)
    plt.title(f"{model.__class__.__name__} Confusion Matrix (Percentages)")
    plt.show()

    return val_accs[-1]


In [ ]:
#Run ALL Models for 5, 15, 30 Epochs
results = []
EPOCH_LIST = [5, 15, 30]

transformer_models = {
    "BERT": (BertTokenizer.from_pretrained("bert-base-uncased"), BertForSequenceClassification, "bert-base-uncased"),
    "DistilBERT": (DistilBertTokenizer.from_pretrained("distilbert-base-uncased"), DistilBertForSequenceClassification, "distilbert-base-uncased"),
    "RoBERTa": (RobertaTokenizer.from_pretrained("roberta-base"), RobertaForSequenceClassification, "roberta-base"),
    "ALBERT": (AlbertTokenizer.from_pretrained("albert-base-v2"), AlbertForSequenceClassification, "albert-base-v2"),
    "XLNet": (XLNetTokenizer.from_pretrained("xlnet-base-cased"), XLNetForSequenceClassification, "xlnet-base-cased")
}

for name, (tok, cls, path) in transformer_models.items():
    for epochs in EPOCH_LIST:
        acc, rep = train_transformer(path, tok, cls, epochs)
        results.append([name, epochs, acc, rep["macro avg"]["f1-score"]])

# LSTM, RNN, NN
for epochs in EPOCH_LIST:
    acc = train_simple_model(LSTMClassifier(), epochs)
    results.append(["LSTM", epochs, acc, None])

    acc = train_simple_model(RNNClassifier(), epochs)
    results.append(["RNN", epochs, acc, None])

    acc = train_simple_model(SimpleNN(), epochs)
    results.append(["SimpleNN", epochs, acc, None])


In [ ]:
df_results = pd.DataFrame(results, columns=[
    "Model", "Epochs", "Accuracy", "MacroF1"
])

df_results


In [ ]:
#improvements

In [ ]:
from transformers import get_linear_schedule_with_warmup

def train_roberta_improved(model_name="roberta-large", epochs=10):

    tokenizer = RobertaTokenizer.from_pretrained(model_name)

    # Increased sequence length
    train_enc = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
    test_enc = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

    train_dataset = EmotionDataset(train_enc, train_labels)
    test_dataset = EmotionDataset(test_enc, test_labels)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=7)
    model = model.to(device)

    # Layer-wise learning rates
    optimizer = torch.optim.AdamW([
        {"params": model.roberta.embeddings.parameters(), "lr": 1e-5},
        {"params": model.roberta.encoder.layer[:12].parameters(), "lr": 2e-5},  # middle layers
        {"params": model.classifier.parameters(), "lr": 5e-5}                    # top layers
    ], weight_decay=0.01)

    total_steps = len(train_loader) * epochs

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )

    scaler = torch.cuda.amp.GradScaler()  # AMP mixed precision

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader):

            optimizer.zero_grad()
            ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            with torch.cuda.amp.autocast():
                output = model(ids, attention_mask=mask, labels=labels)
                loss = output.loss

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            scheduler.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}: Train Loss = {total_loss/len(train_loader):.4f}")

    # Evaluation
    preds, truth = [], []
    model.eval()
    with torch.no_grad():
        for batch in test_loader:
            ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            out = model(ids, attention_mask=mask)
            pred = torch.argmax(out.logits, dim=1)

            preds.extend(pred.cpu().numpy())
            truth.extend(labels.cpu().numpy())

    print("\nClassification Report:")
    print(classification_report(truth, preds))

    acc = accuracy_score(truth, preds)
    print(f"Final Accuracy: {acc:.4f}")

    return acc


In [ ]:
#install dependencies
!pip install transformers torch scikit-learn seaborn --quiet


In [ ]:
#Imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    get_linear_schedule_with_warmup
)

from tqdm import tqdm


In [ ]:
#Dataset Class
class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item


In [ ]:
#Improved RoBERTa-Large Training Code (Best Model)
def train_roberta_large_improved(train_texts, train_labels, test_texts, test_labels,
                                 model_name="roberta-large",
                                 max_len=128,
                                 batch_size=16,
                                 epochs=8):

    print("Using model:", model_name)
    print("Training for", epochs, "epochs")

    tokenizer = RobertaTokenizer.from_pretrained(model_name)

    # Longer sequence length (better for emotion detection)
    train_enc = tokenizer(list(train_texts), truncation=True, padding=True, max_length=max_len)
    test_enc = tokenizer(list(test_texts), truncation=True, padding=True, max_length=max_len)

    train_dataset = EmotionDataset(train_enc, train_labels)
    test_dataset = EmotionDataset(test_enc, test_labels)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=7)
    model = model.to(device)

    # ===== LAYER-WISE LEARNING RATE DECAY =====
    optimizer = torch.optim.AdamW([
        {"params": model.roberta.embeddings.parameters(), "lr": 1e-5},
        {"params": model.roberta.encoder.layer[:12].parameters(), "lr": 2e-5},
        {"params": model.classifier.parameters(), "lr": 5e-5},
    ], weight_decay=0.01)

    # ===== Warmup Scheduler =====
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )

    # ===== Mixed Precision for A100 =====
    scaler = torch.cuda.amp.GradScaler()

    train_losses = []

    # =======================
    #       TRAINING
    # =======================
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            ids = batch['input_ids'].to(device)
            mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            with torch.cuda.amp.autocast():
                outputs = model(ids, attention_mask=mask, labels=labels)
                loss = outputs.loss

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            scheduler.step()

            total_loss += loss.item()

        epoch_loss = total_loss / len(train_loader)
        train_losses.append(epoch_loss)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}")

    # =======================
    #      EVALUATION
    # =======================
    model.eval()
    preds, truth = [], []

    with torch.no_grad():
        for batch in test_loader:
            ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            out = model(ids, attention_mask=mask)
            pred = torch.argmax(out.logits, dim=1)

            preds.extend(pred.cpu().numpy())
            truth.extend(labels.cpu().numpy())

    print("\nClassification Report:")
    print(classification_report(
        truth, preds,
        target_names=['neutral','anger','disgust','fear','happy','sad','surprise']
    ))

    acc = accuracy_score(truth, preds)
    print(f"\nFinal Accuracy: {acc:.4f}")

    # =======================
    #  CONFUSION MATRIX
    # =======================
    cm = confusion_matrix(truth, preds)
    labels = ['neutral','anger','disgust','fear','happy','sad','surprise']

    plt.figure(figsize=(7,6))
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=labels, yticklabels=labels)
    plt.title("Confusion Matrix (Counts)")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

    cm_pct = (cm.astype("float") / cm.sum(axis=1)[:, None]) * 100
    cm_pct = np.round(cm_pct, 2)

    plt.figure(figsize=(7,6))
    sns.heatmap(cm_pct, annot=True, cmap='Greens', fmt='.2f', xticklabels=labels, yticklabels=labels)
    plt.title("Confusion Matrix (Percentages)")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

    return acc


In [ ]:
#run the best model
acc = train_roberta_large_improved(
    train_texts, train_labels,
    test_texts, test_labels,
    model_name="roberta-large",
    epochs=8   # 8 epochs is usually perfect for RoBERTa-large
)

print("Improved Accuracy:", acc)


In [ ]:
#improving bert model from 95 accuracy

In [ ]:
#Install + Imports
!pip install transformers torch scikit-learn seaborn nltk --quiet

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from tqdm import tqdm

import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('omw-1.4')


In [ ]:
#light augmentation
def augment_sentence(sentence, p=0.10):
    words = sentence.split()
    new_words = words.copy()

    for i, w in enumerate(words):
        if np.random.rand() < p and wordnet.synsets(w):
            syns = wordnet.synsets(w)
            lemmas = [l.name().replace("_", " ") for syn in syns for l in syn.lemmas()]
            if lemmas:
                new_words[i] = np.random.choice(lemmas)

    return " ".join(new_words)


In [ ]:
#dataset class
class EmotionDataset(Dataset):
    def __init__(self, enc, labels):
        self.enc = enc
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.enc.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item


In [ ]:
#Improved BERT Training Code (Best Version)
def train_bert_improved(train_texts, train_labels, test_texts, test_labels,
                        model_name="bert-base-uncased",
                        max_len=256,
                        batch_size=16,
                        epochs=10):

    tokenizer = BertTokenizerFast.from_pretrained(model_name)

    # AUGMENT TRAINING DATA LIGHTLY
    aug_texts = []
    aug_labels = []
    for t, l in zip(train_texts, train_labels):
        aug_texts.append(t)
        aug_labels.append(l)

        # add 1 augmented sample
        aug_texts.append(augment_sentence(t))
        aug_labels.append(l)

    # TOKENIZE WITH LONGER SEQUENCES
    train_enc = tokenizer(aug_texts, padding=True, truncation=True, max_length=max_len)
    test_enc  = tokenizer(test_texts, padding=True, truncation=True, max_length=max_len)

    train_dataset = EmotionDataset(train_enc, aug_labels)
    test_dataset  = EmotionDataset(test_enc, test_labels)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # LOAD MODEL
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=7)
    model.to(device)

    # ==========================
    # LAYER-WISE LEARNING RATES
    # ==========================
    optimizer = torch.optim.AdamW([
        {"params": model.bert.embeddings.parameters(), "lr": 1e-5},
        {"params": model.bert.encoder.layer[:8].parameters(), "lr": 2e-5},
        {"params": model.bert.encoder.layer[8:].parameters(), "lr": 3e-5},
        {"params": model.classifier.parameters(), "lr": 5e-5},
    ], weight_decay=0.01)

    total_steps = len(train_loader) * epochs

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )

    scaler = torch.cuda.amp.GradScaler()

    # ==========================
    #       TRAINING LOOP
    # ==========================
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader):
            optimizer.zero_grad()

            ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            with torch.cuda.amp.autocast():
                outputs = model(ids, attention_mask=mask, labels=labels)
                loss = outputs.loss

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(train_loader):.4f}")

    # ==========================
    #       EVALUATION
    # ==========================
    preds, truth = [], []

    model.eval()
    with torch.no_grad():
        for batch in test_loader:
            ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            out = model(ids, attention_mask=mask)
            pred = torch.argmax(out.logits, dim=1)

            preds.extend(pred.cpu().numpy())
            truth.extend(labels.cpu().numpy())

    print("\nClassification Report:")
    print(classification_report(
        truth, preds,
        target_names=['neutral','anger','disgust','fear','happy','sad','surprise']
    ))

    acc = accuracy_score(truth, preds)
    print("\nFINAL IMPROVED BERT ACCURACY:", acc)

    return acc


In [ ]:
#Run the Improved Best Model (BERT)
acc = train_bert_improved(
    train_texts, train_labels,
    test_texts, test_labels,
    epochs=10
)

print("Improved BERT Accuracy:", acc)
